In [0]:
##The code below produces two errors
#Error 1: During training, the accuracy stays around 0.50
#Error 2: The trained model cannot be loaded after training

In [0]:
##Written by Roshan Noronha
##Date: September 3, 2019
##Purpose: The purpose of this notebook is to use transfer learning to improve the accuracy of a convolutional neural network. In this context, any improvement over 79% is considered an improvement.

In [0]:
#link google drive
#the trained model will be saved in google drive to be accessed later
#THIS IS IMPORTANT!
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#get dog and cat pictures from kaggle
#https://www.kaggle.com/general/74235
#command line commands are prefaced with a !

#import kaggle.json
#!pip install -q kaggle
#from google.colab import files
#files.upload()

#kaggle.json is saved in Google Drive
#copy it over to the working directory
!cp /content/drive/My\ Drive/Colab\ Notebooks/hotdognotahotdog/kaggle.json  /content/

In [0]:
#create a kaggle directory and move files the kaggle.json file there
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change permissions of kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#check that kaggle datasets show up
!kaggle datasets list

In [0]:
#get dog and cat pictures from kaggle
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 32.8MB/s]
 98% 265M/271M [00:03<00:00, 60.5MB/s]
100% 271M/271M [00:03<00:00, 82.3MB/s]
 99% 537M/543M [00:07<00:00, 117MB/s]
100% 543M/543M [00:07<00:00, 78.0MB/s]


In [0]:
#unzip training and testing data
!unzip train.zip -d train
!unzip test1.zip -d test

In [0]:
#the training folder has dog and cat images together. These need to be in two seperate folders.
!mkdir train/cats/
!mkdir train/dogs/
!mkdir models/

!find train/train/ | grep "/dog.[0-9]*.jpg" | xargs mv -t train/dogs/
!find train/train/ | grep "/cat.[0-9]*.jpg" | xargs mv -t train/cats/

In [0]:
#train folder is empty so it should be removed
rm -r train/train/

In [0]:
#install gpu-tensorflow and check version
!pip install tensorflow-gpu
import tensorflow as tf
print(tf.__version__)


In [0]:
#import keras and dependancies
#for importing vgg16 use keras.applications NOT keras_applications. Otherwise issues.
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Convolution2D, MaxPooling2D, ZeroPadding2D, Reshape
from keras.preprocessing.image import ImageDataGenerator, image, img_to_array
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D

In [0]:
#show summary of VGG16
#note that the last layer should be popped off here. Keras won't remove the last layer later on.
#the final layer of the created sequential model classifies 1000 different categories
#this layer needs to be removed changed to output just two probabilities
vggmodel = VGG16(weights = "imagenet")
#vggmodel = VGG16(include_top = False, weights = "imagenet")
print(vggmodel.summary())





553467904/553467096 [==============================] - 45s 0us/step


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________________

In [0]:
#create a sequential model and add the vgg16 layers to it
model = Sequential()

for layer in vggmodel.layers:
  model.add(layer)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
#remove the last layer
model.layers.pop()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
#model.pop() does not work
#retry the previous steps but pop the last layer before introducing it into the Sequential model
vggmodel.layers.pop()
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
#last layer gone
#add vggmodel to Sequential again
#create a sequential model and add the vgg16 layers to it
model = Sequential()

for layer in vggmodel.layers:
  model.add(layer)

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
#freeze all the pretrained layers
for layer in model.layers:
  layer.trainable = False


In [0]:
#add a last dense layer
#softmax is used to output a probability
model.add(Dense(1, activation= "softmax"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
#define paths to training and testing folders
train_data = "train/"
test_data = "test/test1/"

#preprocess images
#network asks for images that are 224x224 as input
img_width, img_height = 224,224

#rescale pixel values from [0-255] to [0-1]
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_data, target_size= (img_width, img_height), batch_size= 32, class_mode= 'binary')

Found 25000 images belonging to 2 classes.


In [0]:
#binary_crossentropy is the loss function that measures the inaccuracy of the prediction (backpropagation)
#it is used since we only have two categories
#the optimizer tweaks the weights of all the layers to minimize the error (gradient descent)
model.compile(loss = 'binary_crossentropy', optimizer= 'rmsprop', metrics = ['accuracy'])

#parameters for training the model
#steps is training size / 
model.fit_generator(train_gen, steps_per_epoch= 400, nb_epoch = 4)

#save the weights after model has been trained
model.save("/content/drive/My Drive/Colab Notebooks/hotdognotahotdog/transferlearning-dogsandcats/trainedmodels/V1-transferlearningdogsandcats.h5")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=400, epochs=4)`
  """


Epoch 1/4
400/400 [==============================] - 121s 303ms/step - loss: 8.0223 - acc: 0.4968
Epoch 2/4
400/400 [==============================] - 120s 301ms/step - loss: 7.9388 - acc: 0.5020
Epoch 3/4
400/400 [==============================] - 120s 301ms/step - loss: 7.9799 - acc: 0.4995
Epoch 4/4
400/400 [==============================] - 120s 301ms/step - loss: 7.9226 - acc: 0.5030


In [0]:
#load model
trainedmodel = load_model("/content/drive/My Drive/Colab Notebooks/hotdognotahotdog/transferlearning-dogsandcats/trainedmodels/V1-transferlearningdogsandcats.h5")

ValueError: ignored